In [17]:
def AuAgAlloyDielectricModel(GMF):
    import numpy as np

    c = 2.99792458e17  # Speed of light in nm/s
    h = 4.135667516e-15  # Planck's constant in eV.s

    lambda_vals = np.arange(270, 1201)  # Wavelength range from 270 nm to 1200 nm
    omega = h * c / lambda_vals  # Angular frequency in eV

    # Model parameters
    ModelParameters = np.array([
        [8.9234, 8.5546, 9.0218],  # wp
        [0.042389, 0.022427, 0.16713],  # gammap
        [2.2715, 1.7381, 2.2838],  # einf
        [2.6652, 4.0575, 3.0209],  # wg1
        [2.3957, 3.9260, 2.7976],  # w01
        [0.17880, 0.017723, 0.18833],  # gamma1
        [73.251, 51.217, 22.996],  # A1
        [3.5362, 4.1655, 3.3400],  # w02
        [0.35467, 0.18819, 0.68309],  # gamma2
        [40.007, 30.770, 57.540],  # A2
    ])

    # Compute parameters
    wp = (
        GMF**2 * (2 * ModelParameters[0, 0] - 4 * ModelParameters[0, 2] + 2 * ModelParameters[0, 1])
        + GMF * (-ModelParameters[0, 0] + 4 * ModelParameters[0, 2] - 3 * ModelParameters[0, 1])
        + ModelParameters[0, 1]
    )

    gammap = (
        GMF**2 * (2 * ModelParameters[1, 0] - 4 * ModelParameters[1, 2] + 2 * ModelParameters[1, 1])
        + GMF * (-ModelParameters[1, 0] + 4 * ModelParameters[1, 2] - 3 * ModelParameters[1, 1])
        + ModelParameters[1, 1]
    )

    einf = (
        GMF**2 * (2 * ModelParameters[2, 0] - 4 * ModelParameters[2, 2] + 2 * ModelParameters[2, 1])
        + GMF * (-ModelParameters[2, 0] + 4 * ModelParameters[2, 2] - 3 * ModelParameters[2, 1])
        + ModelParameters[2, 1]
    )

    wg1 = (
        GMF**2 * (2 * ModelParameters[3, 0] - 4 * ModelParameters[3, 2] + 2 * ModelParameters[3, 1])
        + GMF * (-ModelParameters[3, 0] + 4 * ModelParameters[3, 2] - 3 * ModelParameters[3, 1])
        + ModelParameters[3, 1]
    )

    w01 = (
        GMF**2 * (2 * ModelParameters[4, 0] - 4 * ModelParameters[4, 2] + 2 * ModelParameters[4, 1])
        + GMF * (-ModelParameters[4, 0] + 4 * ModelParameters[4, 2] - 3 * ModelParameters[4, 1])
        + ModelParameters[4, 1]
    )

    gamma1 = (
        GMF**2 * (2 * ModelParameters[5, 0] - 4 * ModelParameters[5, 2] + 2 * ModelParameters[5, 1])
        + GMF * (-ModelParameters[5, 0] + 4 * ModelParameters[5, 2] - 3 * ModelParameters[5, 1])
        + ModelParameters[5, 1]
    )

    A1 = (
        GMF**2 * (2 * ModelParameters[6, 0] - 4 * ModelParameters[6, 2] + 2 * ModelParameters[6, 1])
        + GMF * (-ModelParameters[6, 0] + 4 * ModelParameters[6, 2] - 3 * ModelParameters[6, 1])
        + ModelParameters[6, 1]
    )

    w02 = (
        GMF**2 * (2 * ModelParameters[7, 0] - 4 * ModelParameters[7, 2] + 2 * ModelParameters[7, 1])
        + GMF * (-ModelParameters[7, 0] + 4 * ModelParameters[7, 2] - 3 * ModelParameters[7, 1])
        + ModelParameters[7, 1]
    )

    gamma2 = (
        GMF**2 * (2 * ModelParameters[8, 0] - 4 * ModelParameters[8, 2] + 2 * ModelParameters[8, 1])
        + GMF * (-ModelParameters[8, 0] + 4 * ModelParameters[8, 2] - 3 * ModelParameters[8, 1])
        + ModelParameters[8, 1]
    )

    A2 = (
        GMF**2 * (2 * ModelParameters[9, 0] - 4 * ModelParameters[9, 2] + 2 * ModelParameters[9, 1])
        + GMF * (-ModelParameters[9, 0] + 4 * ModelParameters[9, 2] - 3 * ModelParameters[9, 1])
        + ModelParameters[9, 1]
    )

    # Compute Drude term
    Drude = einf - (wp**2) / (omega**2 + 1j * gammap * omega)

    # Compute CP1
    denom = (omega + 1j * gamma1) ** 2

    sqrt_term1 = np.sqrt(omega + 1j * gamma1 - wg1)
    sqrt_term2 = np.sqrt(omega + 1j * gamma1 + wg1)
    sqrt_wg1_w01 = np.sqrt(wg1 - w01)
    sqrt_wg1 = np.sqrt(wg1)

    arg1 = np.sqrt((wg1 - w01) / (omega + 1j * gamma1 - wg1))
    arg2 = np.sqrt((wg1 - w01) / (omega + 1j * gamma1 + wg1))
    arg3 = np.sqrt((wg1 - w01) / wg1)

    term1 = -sqrt_term1 * np.arctan(arg1)
    term2 = -sqrt_term2 * np.arctanh(arg2)
    term3 = 2 * sqrt_wg1 * np.arctanh(arg3)

    First_three_terms = term1 + term2 + term3
    First_three_terms_over_denom = First_three_terms / denom

    fourth_term = (
        sqrt_wg1_w01
        * np.log(1 - ((omega + 1j * gamma1) / w01) ** 2)
        / (2 * denom)
    )

    CP1 = A1 * (First_three_terms_over_denom - fourth_term)

    # Compute CP2
    denom_CP2 = 2 * (omega + 1j * gamma2) ** 2
    numerator_CP2 = np.log(1 - ((omega + 1j * gamma2) / w02) ** 2)
    CP2 = -A2 * numerator_CP2 / denom_CP2

    # Compute Dielectric Function
    DielectricFunction = Drude + CP1 + CP2

    return lambda_vals, DielectricFunction


In [5]:
lambda_vals, DielectricFunction = AuAgAlloyDielectricModel(0.5)
print(lambda_vals, DielectricFunction)

[ 270  271  272  273  274  275  276  277  278  279  280  281  282  283
  284  285  286  287  288  289  290  291  292  293  294  295  296  297
  298  299  300  301  302  303  304  305  306  307  308  309  310  311
  312  313  314  315  316  317  318  319  320  321  322  323  324  325
  326  327  328  329  330  331  332  333  334  335  336  337  338  339
  340  341  342  343  344  345  346  347  348  349  350  351  352  353
  354  355  356  357  358  359  360  361  362  363  364  365  366  367
  368  369  370  371  372  373  374  375  376  377  378  379  380  381
  382  383  384  385  386  387  388  389  390  391  392  393  394  395
  396  397  398  399  400  401  402  403  404  405  406  407  408  409
  410  411  412  413  414  415  416  417  418  419  420  421  422  423
  424  425  426  427  428  429  430  431  432  433  434  435  436  437
  438  439  440  441  442  443  444  445  446  447  448  449  450  451
  452  453  454  455  456  457  458  459  460  461  462  463  464  465
  466 

In [6]:
import numpy as np
import pandas as pd

# Constants
c = 2.99792458e17  # Speed of light in nm/s
h = 4.135667516e-15  # Planck's constant in eV.s

# Wavelength range (converted to nm for calculation, then back to µm for output)
lambda_vals_nm = np.linspace(0.1897 * 1000, 1.9370 * 1000, 500)
omega = h * c / lambda_vals_nm  # Angular frequency in eV

# Example model parameters (same as used in Python code)
GMF = 0.75  # Gold Molar Fraction
ModelParameters = np.array([
    [8.9234, 8.5546, 9.0218],  # wp
    [0.042389, 0.022427, 0.16713],  # gammap
    [2.2715, 1.7381, 2.2838],  # einf
    [2.6652, 4.0575, 3.0209],  # wg1
    [2.3957, 3.9260, 2.7976],  # w01
    [0.17880, 0.017723, 0.18833],  # gamma1
    [73.251, 51.217, 22.996],  # A1
    [3.5362, 4.1655, 3.3400],  # w02
    [0.35467, 0.18819, 0.68309],  # gamma2
    [40.007, 30.770, 57.540],  # A2
])

# Helper function to compute parameters based on GMF
def compute_parameter(idx):
    return (
        GMF**2 * (2 * ModelParameters[idx, 0] - 4 * ModelParameters[idx, 2] + 2 * ModelParameters[idx, 1]) +
        GMF * (-ModelParameters[idx, 0] + 4 * ModelParameters[idx, 2] - 3 * ModelParameters[idx, 1]) +
        ModelParameters[idx, 1]
    )

# Compute model parameters
wp = compute_parameter(0)
gammap = compute_parameter(1)
einf = compute_parameter(2)
wg1 = compute_parameter(3)
w01 = compute_parameter(4)
gamma1 = compute_parameter(5)
A1 = compute_parameter(6)
w02 = compute_parameter(7)
gamma2 = compute_parameter(8)
A2 = compute_parameter(9)

# Drude term
Drude = einf - (wp**2) / (omega**2 + 1j * gammap * omega)

# CP1
denom1 = (omega + 1j * gamma1)**2
CP1 = A1 * (
    -np.sqrt(omega + 1j * gamma1 - wg1) * np.arctan(np.sqrt((wg1 - w01) / (omega + 1j * gamma1 - wg1))) +
    -np.sqrt(omega + 1j * gamma1 + wg1) * np.arctanh(np.sqrt((wg1 - w01) / (omega + 1j * gamma1 + wg1))) +
    2 * np.sqrt(wg1) * np.arctanh(np.sqrt((wg1 - w01) / wg1)) -
    np.sqrt(wg1 - w01) * np.log(1 - ((omega + 1j * gamma1) / w01)**2) / (2 * denom1)
)

# CP2
denom2 = 2 * (omega + 1j * gamma2)**2
CP2 = -A2 * np.log(1 - ((omega + 1j * gamma2) / w02)**2) / denom2

# Dielectric function
DielectricFunction = Drude + CP1 + CP2

# Extract epsilon1 and epsilon2
eps1 = DielectricFunction.real
eps2 = DielectricFunction.imag

# Calculate refractive indices
Re_n = np.sqrt((np.sqrt(eps1**2 + eps2**2) + eps1) / 2)
Im_n = np.sqrt((np.sqrt(eps1**2 + eps2**2) - eps1) / 2)

# Prepare output
lambda_vals_um = lambda_vals_nm / 1000  # Convert nm to µm
data = pd.DataFrame({
    "wave(um)": lambda_vals_um,
    "Re(n)": Re_n,
    "Im(n)": Im_n,
    "eps1": eps1,
    "eps2": eps2
})

# Display the result to the user
# Save to file
file_path = "/mnt/data/AuAg_75Au_25Ag_DielectricData.csv"
data.to_csv(file_path, index=False)
file_path


'/mnt/data/AuAg_75Au_25Ag_DielectricData.csv'

In [14]:
# Define functions for dielectric model and parameters
import numpy as np
def compute_dielectric_parameters(GMF):
    """Compute dielectric model parameters based on GMF."""
    ModelParameters = np.array([
        [8.9234, 8.5546, 9.0218],  # wp
        [0.042389, 0.022427, 0.16713],  # gammap
        [2.2715, 1.7381, 2.2838],  # einf
        [2.6652, 4.0575, 3.0209],  # wg1
        [2.3957, 3.9260, 2.7976],  # w01
        [0.17880, 0.017723, 0.18833],  # gamma1
        [73.251, 51.217, 22.996],  # A1
        [3.5362, 4.1655, 3.3400],  # w02
        [0.35467, 0.18819, 0.68309],  # gamma2
        [40.007, 30.770, 57.540],  # A2
    ])

    def compute_parameter(idx):
        return (
            GMF**2 * (2 * ModelParameters[idx, 0] - 4 * ModelParameters[idx, 2] + 2 * ModelParameters[idx, 1]) +
            GMF * (-ModelParameters[idx, 0] + 4 * ModelParameters[idx, 2] - 3 * ModelParameters[idx, 1]) +
            ModelParameters[idx, 1]
        )

    return {param: compute_parameter(i) for i, param in enumerate(["wp", "gammap", "einf", "wg1", "w01", "gamma1", "A1", "w02", "gamma2", "A2"])}

def compute_dielectric_function(lambda_vals, params):
    """Calculate the dielectric function (eps1 and eps2)."""
    omega = h * c / lambda_vals
    Drude = params["einf"] - (params["wp"]**2) / (omega**2 + 1j * params["gammap"] * omega)
    denom1 = (omega + 1j * params["gamma1"])**2
    CP1 = params["A1"] * (
        -np.sqrt(omega + 1j * params["gamma1"] - params["wg1"]) * 
        np.arctan(np.sqrt((params["wg1"] - params["w01"]) / (omega + 1j * params["gamma1"] - params["wg1"]))) +
        -np.sqrt(omega + 1j * params["gamma1"] + params["wg1"]) * 
        np.arctanh(np.sqrt((params["wg1"] - params["w01"]) / (omega + 1j * params["gamma1"] + params["wg1"]))) +
        2 * np.sqrt(params["wg1"]) * np.arctanh(np.sqrt((params["wg1"] - params["w01"]) / params["wg1"])) -
        np.sqrt(params["wg1"] - params["w01"]) * np.log(1 - ((omega + 1j * params["gamma1"]) / params["w01"])**2) / (2 * denom1)
    )
    denom2 = 2 * (omega + 1j * params["gamma2"])**2
    CP2 = -params["A2"] * np.log(1 - ((omega + 1j * params["gamma2"]) / params["w02"])**2) / denom2
    DielectricFunction = Drude + CP1 + CP2
    return DielectricFunction.real, DielectricFunction.imag

def create_combined_dielectric_file(lambda_vals, GMF, filename):
    """Generate and save combined dielectric data."""
    params = compute_dielectric_parameters(GMF)
    eps1, eps2 = compute_dielectric_function(lambda_vals, params)

    # Calculate refractive indices
    Re_n = np.sqrt((np.sqrt(eps1**2 + eps2**2) + eps1) / 2)
    Im_n = np.sqrt((np.sqrt(eps1**2 + eps2**2) - eps1) / 2)

    combined = pd.DataFrame({
        "wave(um)": lambda_vals / 1000,  # Convert nm to µm
        "Re(n)": Re_n,
        "Im(n)": Im_n,
        "eps1": eps1,
        "eps2": eps2,
    })

    # Write to the specified format
    with open(filename, 'w') as f:
        f.write(f"Rioux calculated dielectric data ({int(GMF*100)}Au/{int((1-GMF)*100)}Ag), Alexander Al-Feghali al-fegha.li\n")
        f.write("1 2 3 0 0 = columns for wave, Re(n), Im(n), eps1, eps2\n")
        f.write("wave(um) Re(n)  Im(n)    eps1   eps2\n")
        combined.to_csv(f, sep='\t', index=False, header=False, float_format="%.6f")

# Generate dielectric files
lambda_vals_nm = np.linspace(0.1897 * 1000, 1.9370 * 1000, 500)  # Wavelength range in nm
# create_combined_dielectric_file(lambda_vals_nm, 100, "75Au25Ag_Rioux_dielec")
# create_combined_dielectric_file(lambda_vals_nm, 0.50, "50Au50Ag_Rioux_dielec")
# create_combined_dielectric_file(lambda_vals_nm, 0.25, "25Au75Ag_Rioux_dielec")


In [18]:
create_combined_dielectric_file(lambda_vals_nm, 100, "100Au_Rioux_test")


C:\Users\alexg\AppData\Local\Temp\ipykernel_49868\3030844772.py:37: RuntimeWarning: invalid value encountered in sqrt
  2 * np.sqrt(params["wg1"]) * np.arctanh(np.sqrt((params["wg1"] - params["w01"]) / params["wg1"])) -
C:\Users\alexg\AppData\Local\Temp\ipykernel_49868\3030844772.py:38: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(params["wg1"] - params["w01"]) * np.log(1 - ((omega + 1j * params["gamma1"]) / params["w01"])**2) / (2 * denom1)
